In [1]:
import argparse
import numpy as np
from scipy.io import wavfile

In [3]:
from audio.processor import WavProcessor, format_predictions

In [2]:
wav_file = "C:\\Users\\Myrna\\Documents\\SensingClues\\devicehive-audio-analysis-master\\wav_files\\elephant3.wav"
sr, data = wavfile.read(wav_file)

In [4]:
proc = WavProcessor()

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt


In [6]:
# Normalize (between 0 and 1)
samples = data / 32768.0

In [7]:
import csv
import os
import tensorflow as tf

from audio import params
from audio.utils import vggish, youtube8m

In [8]:
# Converts audio waveform into an array of examples for VGGish
"""Returns:
      3-D np.array of shape [num_examples, num_frames, num_bands] which represents
      a sequence of examples, each of which contains a patch of log mel
      spectrogram, covering num_frames frames of audio and num_bands mel frequency
      bands, where the frame length is params.STFT_HOP_LENGTH_SECONDS"""
examples_batch = vggish.input.waveform_to_examples(samples, sr)

In [9]:
features = proc._get_features(examples_batch)

In [31]:
predictions = proc._process_features(features)

In [23]:
predictions = proc._filter_predictions(predictions)

In [28]:
type(predictions)

list

In [29]:
len(predictions)

1

In [39]:
predictions.size

527